#TP BLOC 5 : application des réseaux récurrents à la traduction de chaines de caractères

Objectif : des prénoms, représentés sous la forme de chaines de caractères, ont été encryptés par un algorithme inconnu.

Il vous est donné un ensemble de prénoms "d'entrainement" pour lesquels vous disposerez de la chaine de caractère cryptée et de la chaine de caractères décryptée.

Vous devrez apprendre un modèle à base de LSTM capable de décrypter un jeu de prénoms de test pour lequel seul la version cryptée est fournie. Pour les besoins du TP, les chaines de caractères décryptées vous sont également fournies pour le jeu de test afin que vous puissiez vérifier le bon fonctionnement de votre modèle.

---



---



### 1ʳᵉ étape (google colab uniquement)

Si vous souhaitez utiliser google colab, la cellule de code qui suit vous permettra d'accéder à vos fichiers qui se trouvent sur google drive.
Si vous travaillez en local sur votre machine, vous pouvez ignorer cette cellule.**bold text**

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
cur_dir = 'gdrive/My Drive/Colab Notebooks/formation IA niveau 3/module 7 advanced ML/Day2'
# !ls 'gdrive/My Drive/Colab Notebooks/formation IA niveau 3/module 7 advanced ML/Day2'

Mounted at /content/gdrive


### 2nd étape : récupération des données

Les séquences de chaines de caractères sont contenus dans deux fichiers, l'un avec les données d'entrainement, l'autre avec celles de test.

Pour chacun, deux tenseurs sont fournis : le premier contient les séquences 'source' qui sont les séquences cryptées, l'autre les séquences 'target' qui sont décryptés.

Nous vous donnons également une fonction qui permet de transformer les séquences de chiffres en séquences de lettres, plus facile à lire pour les humains.

Toutes les séquences ont été ramenées à 16 caractères par padding. Vous remarquerez aussi qu'il y a un caractère spécial qui marque le début de la chaine.

En regardant les paires de séquences cryptées/décryptées, essayez de comprendre quel code est utilisé pour le cryptage.

En quoi cela justifie-t-il l'usage de réseaux de neurones prenant en compte des données séquentielles ?

In [3]:
import torch
import torch.nn as nn
import numpy as np

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f"Device : {device}")

src_train,tgt_train,num_codes = torch.load(cur_dir+'/6_data_lstm_transformers/'+'train_set.pt') #num_codes = taille alphabet
src_test,tgt_test,num_codes = torch.load(cur_dir+'/6_data_lstm_transformers/'+'test_set.pt')
seq_len = tgt_train.shape[1]


print("taille des tenseurs pour l'entrainement", tgt_train.shape,src_train.shape)
print("taille des tenseurs pour le test", tgt_test.shape,src_test.shape)
print("longueur des séquences : ",seq_len)
def ascii2str(x):
  string =  ''.join([chr(int(i)+ord('a')) if (int(i)+ord('a'))!=ord('z')+2 else '.'  for i in x])
  new_string = string.replace("{", "-" )
  return new_string

for i in range(20):
  print('source : [',ascii2str(src_train[i,:]),"] -> target : [",ascii2str(tgt_train[i,:]),']')



Device : cuda
taille des tenseurs pour l'entrainement torch.Size([2453, 16]) torch.Size([2453, 16])
taille des tenseurs pour le test torch.Size([614, 16]) torch.Size([614, 16])
longueur des séquences :  16
source : [ .tdqdefghijklmno ] -> target : [ .sam------------ ]
source : [ .udvbtfghijklmno ] -> target : [ .taryn---------- ]
source : [ .srfoughijklmnop ] -> target : [ .robin---------- ]
source : [ .tjmqvphijklmnop ] -> target : [ .shiloh--------- ]
source : [ .nhpguehijklmnop ] -> target : [ .melany--------- ]
source : [ .gtbhdxqlijklmno ] -> target : [ .fr-d-ric------- ]
source : [ .srwgsnpzlmnopqr ] -> target : [ .rosaleen------- ]
source : [ .ehttfghijklmnop ] -> target : [ .deon----------- ]
source : [ .mrzltulklmnopqr ] -> target : [ .louella-------- ]
source : [ .mrvwpnijklmnopq ] -> target : [ .lorrie--------- ]
source : [ .ndvowwijklmnopq ] -> target : [ .marion--------- ]
source : [ .xhriltuijklmnop ] -> target : [ .wendell-------- ]
source : [ .taefy-rpz-topqr ] -> targe

### 3eme étape : construction d'un dataset pytorch

Vous allez désormais construire un objet de type 'torch.utils.data.Dataset' dataset compatible avec les "data loader" (torch.utils.data.DataLoader) de pytorch.

Les dernières instructions de la cellule (celles qui vous sont fournies) vous permettra de vérifier que ce que vous avez fait fonctionne.

In [4]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, src_data, tgt_data):
        self.src_data = src_data
        self.tgt_data = tgt_data

    def __len__(self):
        return len(self.src_data)

    def __getitem__(self, idx):
        return self.src_data[idx].long(), self.tgt_data[idx].long()


trainset = CustomDataset(src_train,tgt_train)
testset = CustomDataset(src_test,tgt_test)

train_loader = torch.utils.data.DataLoader(dataset=trainset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=testset, batch_size=16, shuffle=True)

for x_src, x_tgt in test_loader:
  for i in range(x_src.shape[0]):
    print(ascii2str(x_src[i]), '->', ascii2str(x_tgt[i]))
  break

.idqrvvmijklmnop -> .hammond--------
.ed-xvvhijklmnop -> .dawson---------
.kucsefghijklmno -> .ir-n-----------
.xlryvvhijklmnop -> .winton---------
.uhvgukojklmnopq -> .terance--------
.ctisjmwijklmnop -> .brenden--------
.cqhwhghijklmnop -> .andra----------
..xewp-apklmnopq -> .-variste-------
.djuaxhvalklmnop -> .chrysanta------
.kabbefghijklmno -> .izzy-----------
.gg-oujijklmnopq -> .edwina---------
.cojzihijklmnopq -> .aleta----------
.ndxmpndklmnopqr -> .mathieu--------
.i-ohgfghijklmno -> .hylda----------
.edpqh-hijklmnop -> .dallas---------
.krnefghijklmnop -> .joi------------


### 4ᵉ étape : construction du modèle à base de LSTM

Vous construirez ensuite le modèle de LSTM qui permet de faire la traduction de chaines de *caractères*

In [5]:
import torch.nn.functional as F
class my_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(my_LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(num_codes, 20)
        self.lstm = nn.LSTM(input_size=20, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_codes)

    def forward(self, x):
        #Représentation de x choisis par apprentissage avec embedding
        rep_x = self.embedding(x)

        # input of shape (seq_len, batch = N , input_size = 1)
        # output of shape (seq_len, batch, num_directions * hidden_size)
        out, _ = self.lstm(rep_x)

        rep_x2 = torch.reshape(out, (-1,self.hidden_size))

        # Decode hidden state of last time step
        res = self.fc2(F.relu(self.fc1(rep_x2)))
        res = res.reshape(x.shape[0], x.shape[1], -1)

        return res

m = my_LSTM(64,64,2,num_codes)
for x_src, x_tgt in test_loader:
    for i in range(x_src.shape[0]):
        y = m.forward(x_src)
        print(ascii2str(x_src[i]), '->', ascii2str(x_tgt[i]))
    break

.drviltsmlmnopqr -> .cordelia-------
.ehrdefghijklmno -> .den------------
.qtykfghijklmnop -> .prue-----------
.rxnt-rxjklmnopq -> .quintin--------
.ndvonxvoklmnopq -> .marigold-------
.shkoujvoklmnopq -> .reginald-------
.sjiyzfghijklmno -> .rhett----------
.dne-knbcrmnopqr -> .claudette------
.ulqdefghijklmno -> .tim------------
.edvwcsghijklmno -> .darryl---------
.coaxvvhijklmnop -> .alyson---------
.mlpoiwlklmnopqr -> .liliana--------
.m-q-uwmijklmnop -> .lynwood--------
.gns.lzhijklmnop -> .flower---------
.gg-oujijklmnopq -> .edwina---------
.uhvwhvlpklmnopq -> .terrance-------


### 5eme étape : entrainement du modèle

Vous pourrez, par exemple, représenter les caractères par un embedding de taille 20.

In [7]:
"""

à vous d'entrainer le modèle

"""

from tqdm import tqdm


h_dim = 64
net = my_LSTM(h_dim,num_codes,2,num_codes)
optimizer = torch.optim.Adam(net.parameters(), lr=1e-2)
loss_disc = torch.nn.CrossEntropyLoss()
loss = []

for it in tqdm(range(100)):
    l_i = []
    for index, (src, target) in enumerate(train_loader):
        optimizer.zero_grad()
        scores = net(src)
        lg = loss_disc(scores, target.reshape(src.shape[0]*src.shape[1]))
        l_i.append(lg.item())
        lg.backward()
        optimizer.step()
    loss.append(np.mean(l_i))

import pathlib

embedding_size=20
learning_rate = 1e-3
model = my_LSTM(input_size=h_dim,hidden_size=num_codes, num_layers=2, num_classes=num_codes)
model.training = True
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

NUM_EPOCH=50

for epoch in range(NUM_EPOCH):
  global_loss=0
  for x_src, x_tgt in train_loader:
    optimizer.zero_grad()
    x_tgt_pred = model(x_src)
    x_tgt_output = torch.clone(x_tgt[:,:]).reshape(x_src.shape[0]*x_src.shape[1])
    loss = torch.nn.CrossEntropyLoss()(x_tgt_pred.reshape(x_src.shape[0]*x_src.shape[1],-1),x_tgt_output)
    loss.backward()
    optimizer.step()
    global_loss = global_loss+loss.detach().cpu().numpy()
  if epoch%10==0:
      print('epoch: ',epoch, 'loss: ',global_loss)
      print("exemple de décodage d'une donnée de train : ",ascii2str(x_tgt[0,:]),' -> ',ascii2str(torch.argmax(x_tgt_pred,axis=2)[0]),'<fin>')
      my_file = pathlib.Path('./model_transformer2.pt')
      torch.save({'optimizer':optimizer.state_dict(), 'model':model.state_dict()}, my_file)



  0%|          | 0/100 [00:00<?, ?it/s]


ValueError: ignored

### 6eme étape : décodage du jeu de test



In [8]:
model.training = False
for x_src, x_tgt in test_loader:
  y_pred = model(x_src)
  y_pred = torch.argmax(y_pred,axis=2)
  for i in range(x_src.shape[0]):
    print("décodage de [", ascii2str(x_src[i]),"] -> [",ascii2str(y_pred[i]) ,"] GT = ",ascii2str(x_tgt[i]))
    break

NameError: ignored